In [1]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
filepath = "pdf_files/southwindsor_park.pdf"

In [9]:
import os
os.environ["PATH"] = os.getenv('PATH') + ':' + '/opt/homebrew/Cellar/tesseract/5.3.4_1/bin' + ':' + '/opt/homebrew/Cellar/poppler/24.04.0/bin'
print(os.getenv('PATH'))

/Library/Developer/CommandLineTools/usr/bin:/Users/dileepbarla/Library/Python/3.9/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin/opt/homebrew/Cellar/poppler/24.04.0/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/poppler/24.04.0/bin:/opt/homebrew/Cellar/tesseract/5.3.4_1/bin:/opt/homebrew/Cellar/poppler/24.04.0/bin


In [23]:
# Get elements
raw_pdf_elements = partition_pdf(
    filename=filepath,
    
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    # image_output_dir_path="./pdf_outputfiles/",
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [24]:
print(raw_pdf_elements)

[<unstructured.documents.elements.CompositeElement object at 0x3638df250>, <unstructured.documents.elements.CompositeElement object at 0x3613e8760>, <unstructured.documents.elements.Table object at 0x3613e8ee0>, <unstructured.documents.elements.Table object at 0x36432b310>, <unstructured.documents.elements.Table object at 0x358bc41c0>, <unstructured.documents.elements.Table object at 0x17f94f580>, <unstructured.documents.elements.CompositeElement object at 0x36f8bcaf0>, <unstructured.documents.elements.CompositeElement object at 0x36f8bc220>, <unstructured.documents.elements.CompositeElement object at 0x3588c7be0>, <unstructured.documents.elements.CompositeElement object at 0x3588c7bb0>, <unstructured.documents.elements.CompositeElement object at 0x3588c7cd0>, <unstructured.documents.elements.CompositeElement object at 0x3588c7460>, <unstructured.documents.elements.CompositeElement object at 0x3588c7880>, <unstructured.documents.elements.CompositeElement object at 0x3588c7670>, <unstru

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage
import os
from dotenv import load_dotenv
from PIL import Image
import base64
import io

load_dotenv()

True

In [26]:
llm = ChatOpenAI(model="gpt-4-vision-preview")

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [28]:
# Function to convert image to base64
def image_to_base64(image_path):
    with Image.open(image_path) as image:
        buffered = io.BytesIO()
        image.save(buffered, format=image.format)
        img_str = base64.b64encode(buffered.getvalue())
        return img_str.decode('utf-8')

image_str = image_to_base64("figures/figure-1-1.jpg")

In [39]:
chat = ChatOpenAI(model="gpt-4-vision-preview",
                  max_tokens=1024)

msg = chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text" : "Whats the difference between residents and non-residents"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_str}"
                    },
                },
            ]
        )
    ]
)